In [1]:
import pandas as pd
import numpy as np
from glob import glob
import yaml
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

plt.rcParams.update({
    'font.size': 18,
    'font.family': 'serif',
    'axes.linewidth': 1.4,
    'xtick.major.size': 8,
    'ytick.major.size': 8,
    'xtick.minor.size': 4,
    'ytick.minor.size': 4,
    'axes.labelpad': 8,
    'lines.linewidth': 1.8,
    'lines.markersize': 10,
    'axes.titlepad': 15,
    'legend.frameon': True,
    'legend.framealpha': 0.8,
    'xtick.major.width': 2.0,
    'ytick.major.width': 2.0,
    'xtick.minor.width': 1.5,
    'ytick.minor.width': 1.5,
})


In [2]:
def getPaths(indir):
    csvs = glob(indir+'*/'+'*fitparams*.csv')
    paths = sorted(csvs)
    return paths

def getTime(indir,name,period,IDs):
    paths = getPaths(indir)
    median = []
    up = []
    down = []
    ini_path = []
    for path in paths:
        if '.csv' in path:
            now = pd.read_csv(path)
            now.set_index('Parameter',inplace=True)
            t0_name = 't0'
        else:
            ValueError('The file format is not recognized. Please ensure the file is a valid CSV or has the correct format.')

        median.append(float(now.loc[t0_name,'50th']))
        up.append(float(now.loc[t0_name,'+1sigma']))
        down.append(abs(float(now.loc[t0_name,'-1sigma'])))
        ini_path.append(path)
    time = pd.DataFrame({'name':name,'median':median, '+1 sigma':up, '-1 sigma':down})
    # time['median'] += 2400000.5
    time['median'] = [x+2400000.5 if x<2400000.5 else x for x in time['median']]
    time.sort_values(['median'])
    time.index = range(len(time))
    epoch = round((time['median']-time['median'].min())/period)
    time['epoch'] = [int(x) for x in epoch]
    time['ID'] = IDs
    time['path'] = ini_path
    # time.to_csv(indir+'time.csv', index=False)
    # time.to_latex(indir+'time.tex', columns=['median','+1 sigma','-1 sigma','ID'], index=False, float_format="%.7f")
    return time

def binData(data, nbin=100, err=False):
    newdata = np.array(data[:nbin*int(len(data)/nbin)])
    binned = np.nanmean(newdata.reshape(nbin, -1), axis=1)
    if err:
        binned /= np.sqrt(int(len(data)/nbin)-1)
    return binned

def getRps(indir,name,period,IDs):
    paths = getPaths(indir)
    median = []
    up = []
    down = []
    ini_path = []
    for path in paths:
        if '.csv' in path:
            now = pd.read_csv(path)
            now.set_index('Parameter',inplace=True)
            t0_name = 'rp'
        else:
            now = pd.read_csv(path,sep=r'\t\s+', names=['Parameter','50th','+1sigma','-1sigma'], engine='python',skiprows=1)
            now.set_index('Parameter',inplace=True)
            now.index = [x.strip(' ') for x in now.index]
            t0_name = 'rp_p1'

        median.append(float(now.loc[t0_name,'50th']))
        up.append(float(now.loc[t0_name,'+1sigma']))
        down.append(abs(float(now.loc[t0_name,'-1sigma'])))
        ini_path.append(path)
    rps = pd.DataFrame({'name':name,'median':median, '+1 sigma':up, '-1 sigma':down})
    rps.sort_values(['median'])
    rps.index = range(len(rps))
    epoch = round((rps['median']-rps['median'].min())/period)
    rps['epoch'] = [int(x) for x in epoch]
    rps['ID'] = IDs
    rps['path'] = ini_path
    # rps.to_csv(indir+'rps.csv', index=False)
    # rps.to_latex(indir+'rps.tex', columns=['median','+1 sigma','-1 sigma','ID'], index=False, float_format="%.7f")
    return rps


def plotLightCurveAdvanced(axes, nbin, t0s, datas, ids, step=0.02):
    ax, ax1 = axes
    # fig, (ax, ax1) = plt.subplots(1,2, figsize=(24, 12), sharey=True)
    offset = 0
    first = True  # Flag for first iteration
    xpos = min([min(data['t']) for data in datas]) * 0.99
    
    for t0, data, id in zip(t0s, datas, ids):
        center = data['model'].max() - 1
        data['relative flux'] -= center
        data['model'] -= center
        
        labels = {
            'binned': 'Binned Data' if first else None,
            'unbinned': 'Unbinned Data' if first else None,
            'model': 'Best-fit Model' if first else None
        }
        labels_residuals = {
            'binned': 'Binned Residuals' if first else None,
            'unbinned': 'Unbinned Residuals' if first else None,
        }
        binned_flux = binData(data['relative flux'] + offset, nbin)
        binned_t = binData(data['t'], nbin)
        binned_residuals = binData(data['residuals'], nbin)  # No offset for residuals
        binned_error = binData(data['error'], nbin, err=True)
        
        ax.errorbar(binned_t, binned_flux, yerr=binned_error, ms=10, fmt='o',
                    color='deepskyblue', ecolor='deepskyblue', mec='deepskyblue', 
                    zorder=5, label=labels['binned'])
        ax.errorbar(data['t'], data['relative flux'] + offset, yerr=data['error'], ms=10, fmt='o',
                    color='white', ecolor='lightskyblue', mec='lightskyblue', 
                    alpha=0.5, zorder=2, label=labels['unbinned'])
        ax.plot(data['t'], data['model'] + offset, color='dimgray', 
                zorder=10, label=labels['model'])
        
        ax1.errorbar(binned_t, binned_residuals + offset +1, yerr=binned_error, ms=10, fmt='o',
                     color='deepskyblue', ecolor='deepskyblue', mec='deepskyblue', 
                     zorder=5, label=labels_residuals['binned'])
        ax1.errorbar(data['t'], data['residuals'] + offset +1, yerr=data['error'], ms=10, fmt='o',
                     color='white', ecolor='lightskyblue', mec='lightskyblue', 
                     alpha=0.5, zorder=2, label=labels_residuals['unbinned'])
        ax1.plot(data['t'], [offset+1] * len(data['t']), color='dimgray', zorder=10)  # Zero line
        
        ypos = np.mean(np.sort(data['relative flux'])[:int(len(data['relative flux'])/10)]) + 0.55*step + offset    # TESS
        # ypos = np.mean(np.sort(data['relative flux'])[:int(len(data['relative flux'])/10)]) + 0.7*step + offset    # JWST
        ax.text(xpos, ypos, f'{id}\n$t_0 =${t0:.5f}', fontsize=20, 
                ha='left', va='top')
        
        first = False  # Disable labels for subsequent datasets
        offset += step * 1.5

def getLightCurveData(indir,t0):
    files = glob(indir+'*')
    for file in files:
        if '.txt' in file:
            white = pd.read_csv(file,comment='#',sep=' ')
    data = pd.DataFrame({'time':white['time'],'relative flux':white['lcdata'],'error':white['lcerr'],'model':white['model']})
    center = data['model'].max()-1
    data['relative flux'] -= center
    data['model'] -= center
    data['residuals'] = (data['relative flux'] - data['model'])
    # data['t'] = (data['time']-t0+2400000.5)*24
    if np.min(data['time'])>2400000.5:
        data['t'] = (data['time']-t0)*24
    else:
        data['t'] = (data['time']-t0+2400000.5)*24
    
    return data

In [3]:
instruments = ['jwst', 'tess', 'hst']
p = 'WASP-107b'
nbin = 50
fig, axes = plt.subplots(3,2, figsize=(24, 24),height_ratios=[5,2,2] , sharey='row')

for i, ins in enumerate(instruments):
    indir = f'wasp-107b/{ins}/'
    if ins != 'hst':
        with open(f"init_{ins}.yaml", "r") as f:
            init = yaml.safe_load(f)
        name = init[p]['name']
        period = init[p]['period']
        IDs = init[p]['IDs']
        ins_name = init[p]['instrument']
        labels = [f'{id}-{ins}' for id, ins in zip(IDs, ins_name)]
        time = getTime(indir, name, period, IDs)
        # time.to_csv(indir+'time.csv', index=False)

        rps = getRps(indir, name, period, IDs)
        step = np.mean(rps['median'])**2
        datas  =[]
        t0s = []
        ids = []

        for dir, t0, id in zip(['/'.join(s.split('/')[:-1])+'/' for s in time['path']], time['median'], time['ID']):
            datas.append(getLightCurveData(dir,t0))    
            t0s.append(t0)
            ids.append(id)

        
        plotLightCurveAdvanced(axes[i], nbin=nbin, t0s=t0s, datas=datas, ids=labels, step=step)
        ax, ax1 = axes[i]
        ax.set_xlabel('Time from Transit Midpoint [hrs]', fontsize=28)
        
        ax1.set_xlabel('Time from Transit Midpoint [hrs]', fontsize=28)
    else:
        ax, ax1 = axes[i]
        ids = ['14916', '14915']
        t0s = []
        datas = []
        model_datas = []
        for id in ids:
            model_data = pd.read_csv(f'{indir}lc{id}.csv')
            now = np.load(f'{indir}{id}fitting_results.pickle', allow_pickle=True)
            t0 = now['lightcurves']['white']['parameters_final'][-1]
            t0s.append(t0)
            phase = now['lightcurves']['white']['output_time_series']['phase']
            t = (now['lightcurves']['white']['input_time_series']['bjd_tdb'] - t0)*24
            full_model = now['lightcurves']['white']['output_time_series']['full_model']
            full_residuals = now['lightcurves']['white']['output_time_series']['full_residuals']
            systematics = now['lightcurves']['white']['output_time_series']['systematics']
            detrended_lc = now['lightcurves']['white']['output_time_series']['detrended_lc']
            transit = now['lightcurves']['white']['output_time_series']['transit']
            residuals = now['lightcurves']['white']['output_time_series']['residuals']

            data = pd.DataFrame({'t':t,'normalized flux':detrended_lc,'residuals':residuals,'model':transit})
            datas.append(data)

            func = np.poly1d(np.polyfit(phase, t, 1))
            model_data['t'] = func(model_data['phase'])
            model_datas.append(model_data)
            
        xpos = min([min(data['t']) for data in datas]) * 0.99
        step = 0.02
        offset = 0
        first = True

        for t0, data, id, mdata in zip(t0s, datas, ids, model_datas):

            # center = data['model'].max() - 1
            # data['normalized flux'] -= center
            # data['model'] -= center
            # ax.plot(data['t'], data['model'] + offset, color='dimgray', 
            #         zorder=10, label='Best-fit Model' if first else None)
            ax.plot(mdata['t'], mdata['flux_fitted'] + offset, color='dimgray',
                    zorder=10, label='Best-fit Model' if first else None)
            ax.scatter(data['t'], data['normalized flux'] + offset, color='deepskyblue', alpha=0.8, s=120, label='Detrended Data' if first else None)
            
            ax1.scatter(data['t'], data['residuals'] + offset +1, color='deepskyblue', alpha=0.8, s=120)

            ax1.plot(data['t'], [offset+1] * len(data['t']), color='dimgray', zorder=10)  # Zero line

            ypos = np.mean(np.sort(data['normalized flux'])[:int(len(data['normalized flux'])/10)]) + 0.55*step + offset    # TESS
            ax.text(xpos, ypos, f'{id}-HST\n$t_0 =${t0:.5f}', fontsize=20, 
                    ha='left', va='top')
            
            first = False  # Disable labels for subsequent datasets
            offset += step * 1.5

        ax.set_xlabel('Time from Transit Midpoint [hrs]', fontsize=28)
        ax1.set_xlabel('Time from Transit Midpoint [hrs]', fontsize=28)
        # ax.set_xlabel('Phase', fontsize=28)
        # ax1.set_xlabel('Phase', fontsize=28)
    ax.legend(loc='lower right', fontsize=20, frameon=False)
    ax.set_ylabel('Relative Flux', fontsize=28)
    ax.tick_params(direction='in', which='both', labelsize=24)
    ax.minorticks_on()
    ax.xaxis.set_ticks_position('both')
    ax.yaxis.set_ticks_position('both')

    ax1.tick_params(direction='in', which='both', labelsize=24)
    ax1.minorticks_on()
    ax1.xaxis.set_ticks_position('both')
    ax1.yaxis.set_ticks_position('both')
# fig.suptitle(f'White Light Curves and Residuals of {p}', fontsize=36, y=0.98)
plt.tight_layout()
suffix = ''
fig.savefig('white_lightcurves_with_residuals_combined.png', 
            dpi=300, bbox_inches='tight')
fig.savefig('white_lightcurves_with_residuals_combined.pdf', 
            bbox_inches='tight')
fig.savefig('white_lightcurves_with_residuals_transparent_combined.pdf', 
            transparent=True, bbox_inches='tight')
plt.close(fig)